### 6. Zakupoholicy

Dane: https://www.kaggle.com/datasets/benpowis/customer-propensity-to-purchase-data


Cel: analiza czy dany klient wykona u nas zakupy czy nie


Stakeholder: właściciel sklepu internetowego starający się zminimalizować ryzyko
występowania porzuconego koszyka

### CZYSZCZENIE ORAZ ROZDZIELENIE DANYCH

In [1]:
import pandas as pd 

In [2]:
df_train = pd.read_csv('training_sample.csv')

In [3]:
df_train.shape

(455401, 25)

In [4]:
df_train["ordered"].sum()

19093

In [5]:
column_mapping = {
    'UserID': 'ID_uzytkownika',
    'basket_icon_click': 'klik_ikony_koszyka',
    'basket_add_list': 'dodanie_do_koszyka_z_listy',
    'basket_add_detail': 'dodanie_do_koszyka_z_detali',
    'sort_by': 'sortowanie_produktu',
    'image_picker': 'wybor_obrazu',
    'account_page_click': 'klik_na_strone_konta',
    'promo_banner_click': 'klik_na_baner_promo',
    'detail_wishlist_add': 'dodanie_do_listy_zyczen_z_detali',
    'list_size_dropdown': 'rozwijana_lista_produktu',
    'closed_minibasket_click': 'zamkniecie_mini_koszyka',
    'checked_delivery_detail': 'sprawdzenie_info_dostawy_produkt',
    'checked_returns_detail': 'sprawdzenie_info_zwrotu',
    'sign_in': 'logowanie',
    'saw_checkout': 'wyswietlenie_realizacji_zamowienia',
    'saw_sizecharts': 'wyswietlenie_tabeli_rozmiarow',
    'saw_delivery': 'wyswietlenie_info_o_dostawie_FAQ',
    'saw_account_upgrade': 'wyswietlenie_opcji_upgrade_konta',
    'saw_homepage': 'wyswietlenie_strony_glownej',
    'device_mobile': 'urzadzenie_mobilne', 
    'device_computer': 'komputer', 
    'device_tablet': 'tablet',
    'returning_user': 'powracajcy_klient', 
    'loc_uk': 'lokalizacja_uk',
    'ordered': 'zlozone_zamowienie'
}


# zmiana nazw kolumn
df_train = df_train.rename(columns=column_mapping)

In [6]:
# zapis do CSV
df_train.to_csv('training_sample_pl.csv', index=False)

In [7]:
df_train.columns

Index(['ID_uzytkownika', 'klik_ikony_koszyka', 'dodanie_do_koszyka_z_listy',
       'dodanie_do_koszyka_z_detali', 'sortowanie_produktu', 'wybor_obrazu',
       'klik_na_strone_konta', 'klik_na_baner_promo',
       'dodanie_do_listy_zyczen_z_detali', 'rozwijana_lista_produktu',
       'zamkniecie_mini_koszyka', 'sprawdzenie_info_dostawy_produkt',
       'sprawdzenie_info_zwrotu', 'logowanie',
       'wyswietlenie_realizacji_zamowienia', 'wyswietlenie_tabeli_rozmiarow',
       'wyswietlenie_info_o_dostawie_FAQ', 'wyswietlenie_opcji_upgrade_konta',
       'wyswietlenie_strony_glownej', 'urzadzenie_mobilne', 'komputer',
       'tablet', 'powracajcy_klient', 'lokalizacja_uk', 'zlozone_zamowienie'],
      dtype='object')

In [8]:
# lista column do sprawdzenia z zerami
columns_to_check = ['klik_ikony_koszyka', 'dodanie_do_koszyka_z_listy',
       'dodanie_do_koszyka_z_detali', 'sortowanie_produktu', 'wybor_obrazu',
       'klik_na_strone_konta', 'klik_na_baner_promo',
       'dodanie_do_listy_zyczen_z_detali', 'rozwijana_lista_produktu',
       'zamkniecie_mini_koszyka', 'sprawdzenie_info_dostawy_produkt',
       'sprawdzenie_info_zwrotu', 'logowanie',
       'wyswietlenie_realizacji_zamowienia', 'wyswietlenie_tabeli_rozmiarow',
       'wyswietlenie_info_o_dostawie_FAQ', 'wyswietlenie_opcji_upgrade_konta',
       'wyswietlenie_strony_glownej'] 

# 'ID_uzytkownika','urzadzenie_mobilne', 'komputer', 'tablet', 'powracajcy_klient', 'lokalizacja_uk', 'zlozone_zamowienie' - nie jest opisem akcji - jesli = 0 - usuwamy

# filtrowana baza danych gdzie wymienione kolumny = 0
df_train_clean = df_train[~(df_train[columns_to_check] == 0).all(axis=1)]



In [9]:
df_train_clean.shape

(220061, 25)

In [10]:
df_train_clean["zlozone_zamowienie"].sum()

19093

In [11]:
df_train_clean = df_train_clean.sort_values(by='zlozone_zamowienie', ascending= False)

In [12]:
df_train_clean = df_train_clean.reset_index(drop=True)

In [13]:
# lista imion + wydzielenie zbioru testowego
names = ['Ada', 'Darek', 'Grzesiek', 'Ola', 'test']
no_of_orders = df_train_clean["zlozone_zamowienie"].sum()


# funkcja wyboru imiona
def get_name(index):
    return names[index % len(names)]

# dodajemy nowa kolumne - ordered = 1 - wszyscy (z wydzieleniem zbioru testowego), ordered = 0 - po kolei imiona
# df_train_clean['kto'] = df_train_clean.apply(lambda row: 'test' if row['zlozone_zamowienie'] == 1 and row < orders * 0.2 else get_name(row.name), axis=1)

df_train_clean['kto'] = df_train_clean.apply(lambda row: get_name(row.name) if row['zlozone_zamowienie'] == 0 else ('test' if row.name < no_of_orders * 0.2 else 'wszyscy'), axis=1)


In [14]:
df_train_clean['kto'].value_counts()

kto
test        44013
Ola         40194
Ada         40194
Darek       40193
Grzesiek    40193
wszyscy     15274
Name: count, dtype: int64

In [15]:
# Sprawdzamy proporcje zamówień w całym zbiorze df_train_clean oraz w zbiorze wydzilonym do testów

orders_to_total_all = df_train_clean['zlozone_zamowienie'].sum() / df_train_clean['zlozone_zamowienie'].count()
orders_to_total_test = df_train_clean['zlozone_zamowienie'][df_train_clean['kto'] == 'test'].sum() / df_train_clean['zlozone_zamowienie'][df_train_clean['kto'] == 'test'].count()

print(f'odsetek zamówień w całości df_train_clean {orders_to_total_all}')
print(f'odsetek zamówień w test df_train_clean {orders_to_total_test}')

odsetek zamówień w całości df_train_clean 0.0867623068149286
odsetek zamówień w test df_train_clean 0.08676981800831572


In [16]:
# zapis do CSV
df_train_clean.to_csv('training_sample_pl_clean.csv', index=False)

In [17]:
# dodatkowo!!!
# stworzenie bazy danych dla konkretnej osoby - wystarczy zmienic swoje imie

df_train_clean_ola = df_train_clean[df_train_clean['kto'].isin(['wszyscy', 'Ola'])]

In [18]:
df_train_clean_ola['kto'].value_counts()

kto
Ola        40194
wszyscy    15274
Name: count, dtype: int64

In [19]:
# zapis do CSV
df_train_clean_ola.to_csv('training_sample_pl_clean_ola.csv', index=False)

In [20]:
# dodatkowo!!!
# stworzenie bazy danych dla konkretnej osoby - wystarczy zmienic swoje imie

df_test = df_train_clean[df_train_clean['kto'].isin(['test'])]

In [21]:
# zapis do CSV
df_test.to_csv('test_sample_pl.csv', index=False)

In [22]:
df_test['kto'].value_counts()

kto
test    44013
Name: count, dtype: int64